# Introduction:


Problem: There are many people who have restrictions on what they can eat, the reasons for which are numerous. For example some people may be vegatarian, vegan, have religious restrictions, allergic to certain foods, etc and may need a specialty restaurant that can provide meals that satisfy their food requirements. This issue most prominently occurs when people visit a far away city, state or even a country whose food culture is vastly different from the one that they are use to. The problem that I am looking to solve is finding areas within a city that can best accomodate a person's food restrictions/diets/preferences.

Audience: One of the many variables that a person goes through when they are searching for a new home or apartment to live in is the food options available to them whether that is the nearby restaurants they can dine out at or grocery stores that carry the ingrediants that they are looking for. And as mentioned earlier, this information can also be used by people who are visiting a city on vacation and may be worried that their food options will be limited or possibly non-existent. They can then use information like this to prepare and find other areas with restaurants or stores that can fulfill their food needs.

For this project I will focus on a few of the common categories in the city of San Francisco

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

This submission will eventually become your Data section in your final report. So I recommend that you push the report (having your Data section) to your Github repository and submit a link to it.

# Data:

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium

#### Retrieve the latitude, longitude coordinates for San Francisco. By default the Nominatim Geolocator will pinpoint to Downtown San Francisco. But for my project I will need to get different lat,lng coordinates to be able to search different neighbourhoods and areas around San Francisco

In [5]:
address = "San Francisco, CA"

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude, longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


37.7792808 -122.4192363


In [16]:
CLIENT_ID = 'hidden' # your Foursquare ID
CLIENT_SECRET = 'hidden' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: hidden
CLIENT_SECRET:hidden


#### Foursquare API has a 50 return limit on the search query. So in order to get more data I will have to fine tune the search query by changing the lat, lng coordinates, the radius of search, as well as the foursquare ID of the restaurant category(i.e. vegetarian) to get a better idea of the type of food coverage around the city

In [12]:
radius = 1500
LIMIT = 100
search_query = 'Restaurant'
vegetarian_vegan_id = '4bf58dd8d48988d1d3941735'

#Search General list of Restaurants
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION,
    search_query,
    radius, 
    LIMIT)

url

#Search Vegetarian/Vegan Restaurants
v_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION,
    vegetarian_vegan_id,
    radius, 
    LIMIT)

v_url

'https://api.foursquare.com/v2/venues/search?client_id=KNCYXV5LQSPDYOZEKBGQPYDL130HCNE3S5MAHVAGQQP5XDZQ&client_secret=5P3NUVYMEPEHBOILBZHLRGAFK13FBW4SY3LQBH5FNRSW1WEX&ll=37.7792808,-122.4192363&v=20180604&categoryId=4bf58dd8d48988d1d3941735&radius=1500&limit=100'

In [13]:
results = requests.get(url).json()
v_results = requests.get(v_url).json()

In [14]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
processed_df = dataframe[['name', 'location.formattedAddress','location.lat', 'location.lng']]

v_venues = v_results['response']['venues']
v_dataframe = json_normalize(v_venues)
processed_v_df = v_dataframe[['name', 'location.formattedAddress','location.lat', 'location.lng']]

#### The below is a list of restaurants returned by the foursquare API for Downtown San Francisco within a 1500 radius. The data was formatted to list the name and its lat,lng coordinates.

In [10]:
processed_df

,name,location.formattedAddress,location.lat,location.lng
0,Turtle Tower Restaurant,"[645 Larkin St (btwn Eddy & Ellis), San Franci...",37.783989,-122.417764
1,Restaurant & Hotel Upholstery Specialist,"[120 Willow St, San Francisco, CA 94109, Unite...",37.783647,-122.419819
2,Cafe Splash Restaurant,"[750 Ellis St (Btwn Polk and Larkin), San Fran...",37.784162,-122.418519
3,Quintero's Restaurant,[393 Eddy St (btwn Leavenworth St & Wagner Aly...,37.783265,-122.413685
4,Golden Kim Tar Restaurant,[434 Larkin St (btwn Turk St & Golden Gate Ave...,37.781925,-122.417351
5,Thai Idea Vegetarian Restaurant,"[710 Polk St (at Eddy St), San Francisco, CA 9...",37.783446,-122.419307
6,Restaurant Depot,"[San Francisco, CA 94107, United States]",37.776864,-122.419420
7,Restaurant Caliente,"[86 McAllister St, San Francisco, CA 94102, Un...",37.781052,-122.413371
8,9,"[Square HQ (9th Floor), San Francisco, CA, Uni...",37.775177,-122.417993
9,Hai's Restaurant,"[227 Hyde St (btwn Eddy & Turk), San Francisco...",37.783001,-122.415808


#### I narrowed down the search to "Vegetarian / Vegan" restaurants (Foursquare does not differentiate between Vegetarian / Vegan) and this search resulted in the following list of restaurants located in downtown SF. This indicates that Downtown San Francisco has a good amount of  of vegetarian / vegan options making it an easy visit for tourist vegetarians / vegans within a 1500 radius of the nomanatim lat lng coordinates

In [15]:
processed_v_df

,name,location.formattedAddress,location.lat,location.lng
0,Ananda Fuara,"[1298 Market St (at Larkin St), San Francisco,...",37.777693,-122.416353
1,Golden Era Vegan,"[395 Golden Gate Ave, San Francisco, CA 94102,...",37.781495,-122.416822
2,Vegan Picnic Polk St,"[1323 Polk St, San Francisco, CA 94109, United...",37.788959,-122.420448
3,Hella Vegan Eats,"[On the go!, California, United States]",37.770233,-122.414000
4,Vegan Plaza,"[995 Market St (6th street), San Francisco, CA...",37.782229,-122.410358
5,Core Kitchen (@ the Hall),"[San Francisco, CA, United States]",37.781810,-122.411212
6,Martin De Porres House of Hospitality,"[225 Potrero Ave, San Francisco, CA 94103, Uni...",37.766641,-122.407233
7,Shizen,"[370 14th St (at Stevenson St), San Francisco,...",37.768190,-122.421627
8,Lydia's Lovin' Foods,"[San Francisco, CA, United States]",37.779406,-122.416951
9,Mixt,"[51 Yerba Buena Ln, San Francisco, CA 94103, U...",37.785751,-122.404124
